In [6]:
# Dependencies
import requests
import pandas as pd

import matplotlib.pyplot as plt

In [11]:
# Define URL parameters
league_id = "730129"
year = 2022
week = 1

In [8]:
# Define cookie parameters - hide these later
private keys here, more detail later

In [25]:
# Define the URL with our parameters
url = f"https://fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{league_id}"
#'http://games.espn.com/ffl/api/v2/scoreboard'

In [26]:
# Pull team and matchup data from the URL
response = requests.get(url, 
                                params={"leagueId" : league_id,
                                       "seasonId" : year,
                                       "matchupPeriodId" : week,
                                       "view": "mMatchup"}, 
                                verify=False,
                               cookies={"swid" : swid,
                                       "espn_s2" : espn_s2},)

C:\Users\bryantim\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fantasy.espn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [27]:
response.json()

{'draftDetail': {'drafted': True, 'inProgress': False},
 'gameId': 1,
 'id': 730129,
 'schedule': [{'away': {'cumulativeScore': {'losses': 0,
     'scoreByStat': {'128': {'ineligible': False,
       'rank': 0.0,
       'result': None,
       'score': 0.0},
      '1': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 14.0},
      '129': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 0.0},
      '130': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 1.0},
      '3': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 134.0},
      '4': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 0.0},
      '133': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 0.0},
      '134': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 0.0},
      '135': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 0.0},
      '136': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 0.0},
      '16': {'ineligible': False, '

In [ ]:
json